In [2]:
import numpy as np
import pandas as pd
import re
import emoji
import inflect
from tqdm import tqdm
import copy
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
import pickle
from sentence_transformers import SentenceTransformer,util
from sentence_transformers import SentencesDataset, LoggingHandler, losses
from sklearn.metrics import classification_report


/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/iLab-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
list(data[data.target==0].text)

['"Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznnPNi8',
 'If this child was Chinese, this tweet would have gone viral. Social media would be ablaze. SNL would have made a racist j…',
 "National Security Minister, Kan Dapaah's side chic has set the internet ablaze with her latest powerful video.… https://t.co/rhzOMQVSlj",
 'This creature who’s soul is no longer clarent but blue ablaze This thing Carrying memories Memories of… https://t.co/tBKSNDrDoX',
 'Social media went bananas after Chuba Hubbard announced Monday evening his plans to return to #okstate. https://t.co/peN…',
 'AMEN! Set the whole system ablaze, man. https://t.co/J08xHDcGbD',
 'Rengoku sets my heart ablaze😔❤️🔥 P.s. I missed this style of coloring I do so here it is c: #鬼滅の刃 https://t.co/YrUF9g68s0',
 'paulzizkaphoto: “Rundle Ablaze” Wishing you all a good evening... https://t.co/d0NlME1HQz https://t.co/hlVlT6qiIp',
 '#ThankfulTuesday

In [3]:
data = pd.read_csv('/Users/savinaysingh/Downloads/tweets 2.csv')


In [4]:
data = data[['text','target']]


In [5]:
data.isna().sum()

text      0
target    0
dtype: int64

# 1. Capturing Hashtags

In [6]:
pattern = r'#\w+'

# Extract and capture hashtags in a new column 'hashtags'
data['hashtags'] = data['text'].str.findall(pattern)


In [7]:
sentences=data['text']

# 2. Handling Apostophe

In [8]:
number_sent_apostophe=len([x for x in sentences if "'" in x])

In [9]:
print("Percentage of sentences with apostophe is: ",number_sent_apostophe/len(sentences)*100,"%")

Percentage of sentences with apostophe is:  16.710642040457344 %


In [10]:
# Converting the words having apostophe into their root form
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"there\'s", "there is", phrase)
    phrase = re.sub(r"it\'s", "it is", phrase)
    phrase = re.sub(r"he\'s", "he is", phrase)
    phrase = re.sub(r"she\'s", "she is", phrase)
    phrase = re.sub(r"how\'s", "how is", phrase)
    phrase = re.sub(r"let\'s", "let is", phrase)
    phrase = re.sub(r"so\'s", "so is", phrase)
    phrase = re.sub(r"what\'s", "what is", phrase)
    phrase = re.sub(r"when\'s", "when is", phrase)
    phrase = re.sub(r"where\'s", "where is", phrase)
    phrase = re.sub(r"why\'s", "why is", phrase)
  
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [11]:
[x for x in sentences if "won't" in x]

["People are going bankrupt over medical debt. $12k a year minus the VAT won't put a dent in that. Let'… https://t.co/i31T5Kg5LE",
 "There's a new drug in town and you won't want to miss its catastrophic effects on tonight's all-new NCIS: Los Angeles. Stream i…",
 '"Let me tell you something good... Beel won\'t destroy the human world, Do you know why? It\'s because I\'m his parent… https://t.co/H5iBsk2BxX',
 "Translation: we have a climate policy but that won't stop us from investing in things that destroy the climate, because maki…",
 "So long as I keep moving, play loud music to drown him, play videogames or watch TV shows or movies, I won't have t… https://t.co/fJpK9wA2W6",
 "Why won't my blue_aura_Explosion spawn!?!?!? Because the naming convention is aura_blue_Explosion ... #indiedev https:…",
 "The story that won't die. 2 more dogs. Cane Corsos. https://t.co/7oQxpwqIKh",
 "despite immature fans wanting him gone, I don't have the fear that jongdae will leave bc I know exo won't L

In [12]:
sentences=[decontracted(x) for x in sentences]

In [13]:
[x for x in sentences if "won't" in x]

[]

In [14]:
number_sent_apostophe=len([x for x in sentences if "'" in x])

In [15]:
print("Percentage of sentences with apostophe is: ",number_sent_apostophe/len(sentences)*100,"%")

Percentage of sentences with apostophe is:  8.988566402814424 %


In [16]:
data['text'] =sentences

# 3. Lowercasing

In [17]:
data['text'] = [entry.lower() for entry in data['text']]

# 4. Removing Emojis, URL, hashtags and spl chars

In [18]:
# De-emojize the text
def remove_emojis(text):
    return emoji.demojize(text)

# url not req while finding if text is disaster or not
def remove_urls(text):
    pattern = re.compile(r'http\S+|www\S+')
    return pattern.sub('', text)

def remove_hashtags(text):
    pattern = re.compile(r'#\w+')
    return pattern.sub('', text)

# spl chars not req while finding if text is disaster or not
def remove_special_characters(text):
    pattern = re.compile(r'[^a-zA-Z0-9\s]')
    return pattern.sub('', text)

In [19]:
def preprocess_text(text):
    text = remove_urls(text)
    text = remove_hashtags(text)
    text = remove_special_characters(text)
    text = remove_emojis(text)
    return text

In [20]:
data['text'] = data['text'].apply(preprocess_text)


In [21]:
# Imbalanced Dataset
data.target.value_counts()

target
0    9256
1    2114
Name: count, dtype: int64

In [22]:
# Decision on numbers

In [23]:
pd.Series([[w for w in d.split() if w.isdigit()] for d in data.text]).value_counts()

[]                       9341
[2]                       106
[1]                        79
[2020]                     62
[5]                        54
                         ... 
[47, 3]                     1
[22000]                     1
[6150000000, 1981, 0]       1
[6150000000, 1981]          1
[222]                       1
Name: count, Length: 770, dtype: int64

In [24]:
print((1-9331/len(data))*100,'% tweets have digits',sep='')

17.933157431838175% tweets have digits


In [25]:
[d for d in data.text if '144' in d]

['telangana section 144 has been imposed in bhainsa from january 13 to 15 after clash erupted between two groups on january 12 po',
 '1441 earthquakes in puerto rico in the last 30 days 1441 the earth keeps shaking families are sleeping on the st',
 '1144fatality sr41  hanford armona rd chp hanford area ']

In [26]:
# Numbers seem important

# 5. Handling Numerics

In [27]:
p = inflect.engine()

In [28]:
sentences=list(data.text)

In [29]:
# Convering all the digits into their respective text i.e 19-> nineteen 
final_ans=[]
for i,sent in tqdm(enumerate(sentences),total=len(sentences)):
    list_numbers=re.findall(r'[0-9]+', sent)
    list_numbers=[int(l) for l in list_numbers]
    list_words=[p.number_to_words(l)  for l in list_numbers]
    dict_NW=dict(zip(list_numbers,list_words))
    myKeys = list(dict_NW.keys())
    myKeys.sort(reverse=True)
    sorted_dict = {i: dict_NW[i] for i in myKeys}
    ans=copy.copy(sent)
    for key in sorted_dict.keys():
        ans=re.sub(str(key),' '+sorted_dict[key]+' ',ans)
    final_ans.append(ans)

100%|██████████████████████████████████| 11370/11370 [00:00<00:00, 80398.74it/s]


In [30]:
sentences=final_ans.copy()

In [31]:
data.text=sentences

In [32]:
def removeDash(input_string):
    return re.sub(r'-', ' ', input_string)


In [33]:
data['text']=data['text'].apply(removeDash)

In [34]:
data.text.nunique()

10938

In [35]:
data

,text,target,hashtags
0,communal violence in bhainsa telangana stones ...,1,[]
1,telangana section one hundred and forty four ...,1,[]
2,arsonist sets cars ablaze at dealership,1,[]
3,arsonist sets cars ablaze at dealership,1,[]
4,lord jesus your love brings freedom and pardon...,0,[]
...,...,...,...
11365,media should have warned us well in advance th...,0,[]
11366,i feel directly attacked i consider moonbin a...,0,[]
11367,i feel directly attacked i consider moonbin a...,0,[]
11368,ok who remember outcast nd the dora au those a...,0,[]


# 6. Removing Duplicate Tweets

In [36]:
# Removal of duplicates
data=data.drop_duplicates(subset=['text'])

In [37]:
data

,text,target,hashtags
0,communal violence in bhainsa telangana stones ...,1,[]
1,telangana section one hundred and forty four ...,1,[]
2,arsonist sets cars ablaze at dealership,1,[]
3,arsonist sets cars ablaze at dealership,1,[]
4,lord jesus your love brings freedom and pardon...,0,[]
...,...,...,...
11365,media should have warned us well in advance th...,0,[]
11366,i feel directly attacked i consider moonbin a...,0,[]
11367,i feel directly attacked i consider moonbin a...,0,[]
11368,ok who remember outcast nd the dora au those a...,0,[]


In [38]:
data.target.value_counts()

target
0    8991
1    1947
Name: count, dtype: int64

In [39]:
# Hashtag cleaing
data.hashtags.value_counts()

hashtags
[]                             9490
[#StormBrendan]                  18
[#Iran]                          11
[#TaalVolcano]                   10
[#Hitchin]                        8
                               ... 
[#icymi]                          1
[#teamworking, #demolition]       1
[#RT, #BYNR, #inspy]              1
[#DemDebate, #IowaCau]            1
[#WWII]                           1
Name: count, Length: 1189, dtype: int64

In [40]:
data.hashtags=['' if [] == d else ' '.join([remove_special_characters(w) for w in d]) for d in data.hashtags]

/var/folders/0h/j84rsdwd49jbw1_d04_c5b3r0000gn/T/ipykernel_20548/4157048724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.hashtags=['' if [] == d else ' '.join([remove_special_characters(w) for w in d]) for d in data.hashtags]


In [41]:
data

,text,target,hashtags
0,communal violence in bhainsa telangana stones ...,1,
1,telangana section one hundred and forty four ...,1,
2,arsonist sets cars ablaze at dealership,1,
3,arsonist sets cars ablaze at dealership,1,
4,lord jesus your love brings freedom and pardon...,0,
...,...,...,...
11365,media should have warned us well in advance th...,0,
11366,i feel directly attacked i consider moonbin a...,0,
11367,i feel directly attacked i consider moonbin a...,0,
11368,ok who remember outcast nd the dora au those a...,0,


In [42]:
data.target.value_counts()

target
0    8991
1    1947
Name: count, dtype: int64

In [43]:
sentences=data.text

# 7. Lemmatization

In [44]:
def lemmatize_text(preprocessed_text):
    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        # print(tag)
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
   # 2. Lemmatize a Sentence with the appropriate POS tag
    lemmatized_text=[lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(preprocessed_text)]
    # The final processed set of words for each iteration will be stored in 'text_final'
    text_final=(" ".join( lemmatized_text ))
    return text_final

In [45]:
ans=[]
for x in tqdm(sentences,total=len(sentences)):
    ans.append(lemmatize_text(x))

100%|████████████████████████████████████| 10938/10938 [00:18<00:00, 592.32it/s]


In [46]:
sentences=ans.copy()

In [47]:
data.text=sentences

/var/folders/0h/j84rsdwd49jbw1_d04_c5b3r0000gn/T/ipykernel_20548/3771718101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text=sentences


# 8. Stopwords Removal

In [48]:
# stopword removal

In [49]:
stop_words=stopwords.words('english')

In [50]:
sentences=[' '.join([word for word in x.split() if word not in stop_words]) for x in sentences]

In [51]:
data.text=sentences

/var/folders/0h/j84rsdwd49jbw1_d04_c5b3r0000gn/T/ipykernel_20548/3771718101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.text=sentences


In [52]:
data.text

0        communal violence bhainsa telangana stone pelt...
1        telangana section one hundred forty four impos...
2                       arsonist set car ablaze dealership
3                       arsonist set car ablaze dealership
4        lord jesus love brings freedom pardon fill hol...
                               ...                        
11365    medium warn u well advance wreck whole night r...
11366    feel directly attack consider moonbin amp jinj...
11367    feel directly attack consider moonbin amp jinj...
11368    ok remember outcast nd dora au au wreck nerve ...
11369                jake corway wreck run fourteen th irp
Name: text, Length: 10938, dtype: object

# 9. Classification

## 9.1. Count Vectorizer

In [146]:
train_size = 0.6
test_size = 0.2
dev_size = 0.2

df_train, temp_df = train_test_split(data, test_size=(test_size + dev_size), random_state=42)
df_test, df_dev = train_test_split(temp_df, test_size=(dev_size / (test_size + dev_size)), random_state=42)


In [147]:
# max_df: ignore terms that have a document frequency strictly higher than 0.09
# min_df: ignore terms that have a document frequency strictly lower than 0.001
# ngram_range: Take unigram,bigrams and trigrams
vectorizer = CountVectorizer(ngram_range=(1,3))
vectorizer2 = CountVectorizer(ngram_range=(1,3))

In [148]:
matrix = vectorizer.fit_transform(df_train['text'])

In [149]:
countv_train=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [150]:
matrix2 = vectorizer2.fit_transform(df_train['hashtags'])

In [151]:
countv_train2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [152]:
%pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [153]:
labels_train=df_train.target

In [154]:
countv_train_3=pd.concat([countv_train, countv_train2], axis=1)


In [157]:
smote = SMOTE(sampling_strategy='auto', random_state=42)


In [158]:
X_resampled, y_resampled = smote.fit_resample(countv_train_3, labels_train)


In [163]:
y_resampled.value_counts()

target
0    5422
1    5422
Name: count, dtype: int64

In [164]:
rf_classifier=RandomForestClassifier(n_estimators=133,min_samples_split=30,max_depth=140,random_state=15)
rf_classifier.fit(X_resampled,y_resampled)

RandomForestClassifier(max_depth=140, min_samples_split=30, n_estimators=133,
                       random_state=15)

In [165]:
pred_train=rf_classifier.predict(X_resampled)

In [166]:
print('Training accuracy is: ',accuracy_score(y_resampled,pred_train)*100)

Training accuracy is:  91.24861674658797


In [167]:
# Validation Set
matrix = vectorizer.transform(df_dev['text'])
countv_dev=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [168]:
matrix2 = vectorizer2.transform(df_dev['hashtags'])
countv_dev2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [169]:
countv_dev_3=pd.concat([countv_dev, countv_dev2], axis=1)


In [170]:
labels_dev=df_dev.target

In [171]:
pred_dev=rf_classifier.predict(countv_dev_3)

In [172]:
print('Validation accuracy is: ',accuracy_score(labels_dev,pred_dev)*100)

Validation accuracy is:  76.96526508226691


In [173]:
# Testing Set
matrix = vectorizer.transform(df_test['text'])
countv_test=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [174]:
matrix2 = vectorizer2.transform(df_test['hashtags'])
countv_test2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [175]:
countv_test_3=pd.concat([countv_test, countv_test2], axis=1)


In [176]:
labels_test=df_test.target

In [177]:
pred_test=rf_classifier.predict(countv_test_3)

In [178]:
print('Testing accuracy is: ',accuracy_score(labels_test,pred_test)*100)

Testing accuracy is:  76.05118829981718


In [179]:
precision = precision_score(labels_test,pred_test)
recall = recall_score(labels_test,pred_test)
f1 = f1_score(labels_test,pred_test)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.348643006263048
Recall: 0.44063324538258575
F1-score: 0.3892773892773893


## 9.2. TFIDF

In [180]:
train_size = 0.6
test_size = 0.2
dev_size = 0.2

df_train, temp_df = train_test_split(data, test_size=(test_size + dev_size), random_state=42)
df_test, df_dev = train_test_split(temp_df, test_size=(dev_size / (test_size + dev_size)), random_state=42)


In [181]:
# max_df: ignore terms that have a document frequency strictly higher than 0.09
# min_df: ignore terms that have a document frequency strictly lower than 0.001
# ngram_range: Take unigram,bigrams and trigrams
vectorizer = TfidfVectorizer(ngram_range=(1,3))
vectorizer2 = TfidfVectorizer(ngram_range=(1,3))

In [182]:
matrix = vectorizer.fit_transform(df_train['text'])

In [183]:
countv_train=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [184]:
matrix2 = vectorizer2.fit_transform(df_train['hashtags'])

In [185]:
countv_train2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [186]:
df_train

,text,target,hashtags
3291,hubris pronounce like debris pronounce like,0,
10094,u official involve embassy security say notify...,1,
4306,use resource build pipeline water rich fnq dro...,0,
11077,drawing people,0,
7144,two thousand thirteen roulot luchets meursault...,0,
...,...,...,...
5924,book hear amp discus fifteen mar,0,Floods RisingWaters Essex AllAtSea
5350,malaysia master winner suffer cut wholebody br...,1,FMTNews Badminton
5558,point surely reach top point penny start drop ...,0,
886,"one thousand , six hundred two mile teach gun ...",0,


In [187]:
labels_train=df_train.target

In [188]:
countv_train_3=pd.concat([countv_train, countv_train2], axis=1)


In [189]:
X_resampled, y_resampled = smote.fit_resample(countv_train_3, labels_train)


/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/iLab-env/lib/python3.8/site-packages/threadpoolctl.py:1019: RuntimeWarning: libc not found. The ctypes module in Python 3.8 is maybe too old for this OS.
  warnings.warn(


In [190]:
# 1
rf_classifier=RandomForestClassifier(n_estimators=133,min_samples_split=30,max_depth=140,random_state=15)
rf_classifier.fit(X_resampled,y_resampled)

RandomForestClassifier(max_depth=140, min_samples_split=30, n_estimators=133,
                       random_state=15)

In [224]:
# 2
rf_classifier=RandomForestClassifier(n_estimators=133,min_samples_split=30,max_depth=140,random_state=15,class_weight={0:1,1:2})
rf_classifier.fit(X_resampled,y_resampled)


RandomForestClassifier(class_weight={0: 1, 1: 2}, max_depth=140,
                       min_samples_split=30, n_estimators=133, random_state=15)

In [241]:
# 3
rf_classifier=RandomForestClassifier(n_estimators=133,min_samples_split=30,max_depth=140,random_state=15,class_weight={0:1,1:3})
rf_classifier.fit(X_resampled,y_resampled)


RandomForestClassifier(class_weight={0: 1, 1: 3}, max_depth=140,
                       min_samples_split=30, n_estimators=133, random_state=15)

In [242]:
pred_train=rf_classifier.predict(X_resampled)

In [243]:
print('Training accuracy is: ',accuracy_score(y_resampled,pred_train)*100)

Training accuracy is:  98.64441165621541


In [244]:
# Validation Set
matrix = vectorizer.transform(df_dev['text'])
countv_dev=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [245]:
matrix2 = vectorizer2.transform(df_dev['hashtags'])
countv_dev2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [246]:
countv_dev_3=pd.concat([countv_dev, countv_dev2], axis=1)


In [247]:
labels_dev=df_dev.target

In [248]:
pred_dev=rf_classifier.predict(countv_dev_3)

In [249]:
print('Validation accuracy is: ',accuracy_score(labels_dev,pred_dev)*100)

Validation accuracy is:  83.77513711151737


In [250]:
# Testing Set
matrix = vectorizer.transform(df_test['text'])
countv_test=pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())

In [251]:
matrix2 = vectorizer2.transform(df_test['hashtags'])
countv_test2=pd.DataFrame(matrix2.toarray(),columns=vectorizer2.get_feature_names_out())

In [252]:
countv_test_3=pd.concat([countv_test, countv_test2], axis=1)


In [253]:
labels_test=df_test.target

In [254]:
pred_test=rf_classifier.predict(countv_test_3)

In [255]:
print('Testing accuracy is: ',accuracy_score(labels_test,pred_test)*100)

Testing accuracy is:  83.82084095063985


In [256]:
precision = precision_score(labels_test,pred_test)
recall = recall_score(labels_test,pred_test)
f1 = f1_score(labels_test,pred_test)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.5242718446601942
Recall: 0.712401055408971
F1-score: 0.6040268456375839


In [257]:
# We need more recall

In [261]:
with open('/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/rf_classifier.pkl', 'wb') as file:
    pickle.dump(rf_classifier, file)

In [262]:
with open('/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/vectorizer_tweet.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

In [263]:
with open('/Users/savinaysingh/Documents/SavinayUTS/iLab/Code/vectorizer_hashtag.pkl', 'wb') as file:
    pickle.dump(vectorizer2, file)

# Same train test split

# Now training on complete dataset

# Attempt for Contextual model

In [268]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [274]:
tweets=list(data.text)

In [277]:
tweet_embeddings = model.encode(tweets)

In [282]:
hashtags=list(data.hashtags)

In [283]:
hashtag_embeddings = model.encode(hashtags)

In [284]:
hashtag_embeddings.shape

(10938, 768)

In [285]:
tweet_embeddings.shape

(10938, 768)

In [286]:
concatenated_embedding = np.concatenate((tweet_embeddings, hashtag_embeddings), axis=1)


In [290]:
labels=data.target

In [287]:
concatenated_embedding.shape

(10938, 1536)

In [302]:
X_train, X_test, y_train, y_test = train_test_split(concatenated_embedding, labels, test_size=0.2, random_state=42)


In [303]:
X_train, y_train = smote.fit_resample(X_train, y_train)


In [304]:
y_train.value_counts()

target
1    7217
0    7217
Name: count, dtype: int64

In [319]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [320]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [321]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8807129798903108


In [322]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.7297297297297297
Recall: 0.5869565217391305
F1-score: 0.6506024096385542


In [323]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,class_weight={0:1,1:1.5})


In [324]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 1, 1: 1.5}, random_state=42)

In [325]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8743144424131627


In [326]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.7278688524590164
Recall: 0.5362318840579711
F1-score: 0.6175243393602227


In [327]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,class_weight={0:1,1:0.5})


In [328]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 1, 1: 0.5}, random_state=42)

In [329]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.879798903107861


In [330]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.6981627296587927
Recall: 0.642512077294686
F1-score: 0.6691823899371069


In [331]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,class_weight={0:1.5,1:1})


In [332]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 1.5, 1: 1}, random_state=42)

In [333]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.879798903107861


In [334]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.7150997150997151
Recall: 0.606280193236715
F1-score: 0.6562091503267974


In [335]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,class_weight={0:2,1:1})


In [336]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 2, 1: 1}, random_state=42)

In [337]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.879798903107861


In [338]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.6981627296587927
Recall: 0.642512077294686
F1-score: 0.6691823899371069


In [353]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42,class_weight={0:5,1:1})


In [354]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(class_weight={0: 5, 1: 1}, random_state=42)

In [355]:

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8638025594149908


In [356]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.6228813559322034
Recall: 0.7101449275362319
F1-score: 0.6636568848758465


In [343]:
precision = precision_score(labels_test,pred_test)
recall = recall_score(labels_test,pred_test)
f1 = f1_score(labels_test,pred_test)

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.5242718446601942
Recall: 0.712401055408971
F1-score: 0.6040268456375839


In [347]:
report = classification_report(labels_test,pred_test)

# Print the report
print(report)

              precision    recall  f1-score   support

           0       0.93      0.86      0.90      1809
           1       0.52      0.71      0.60       379

    accuracy                           0.84      2188
   macro avg       0.73      0.79      0.75      2188
weighted avg       0.86      0.84      0.85      2188



In [348]:
report = classification_report(y_test, y_pred)

# Print the report
print(report)

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1774
           1       0.68      0.66      0.67       414

    accuracy                           0.88      2188
   macro avg       0.80      0.79      0.80      2188
weighted avg       0.88      0.88      0.88      2188



In [357]:
report = classification_report(y_test, y_pred)

# Print the report
print(report)

              precision    recall  f1-score   support

           0       0.93      0.90      0.91      1774
           1       0.62      0.71      0.66       414

    accuracy                           0.86      2188
   macro avg       0.78      0.80      0.79      2188
weighted avg       0.87      0.86      0.87      2188

